In [2]:
import time
import os
import json
import pandas as pd
import numpy as np
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from itertools import chain

In [3]:
# PATHs
# The directory where the raw transaction and punk data are stored
ORI_DATA_PATH = '../data/ori'

# The directory where the databases are stored
DATABASE_PATH = '../data/database'
if os.path.exists(DATABASE_PATH) is False:
    os.makedirs(DATABASE_PATH)

IMG_PATH = '../img'

In [269]:
tx_db = pd.read_csv(f'{DATABASE_PATH}/tx_db.csv', index_col=0)
tx_db.sort_values(by='date', inplace=True)

punk_db = pd.read_csv(f'{DATABASE_PATH}/punk_db.csv', index_col=0)
punk_db['attributes'] = punk_db['attributes'].apply(eval)

attributes_unique_list = set(list(chain(*list(punk_db.attributes))))

address_dict = json.load(open('{}/addresses.json'.format(DATABASE_PATH)))
address_dict = {int(k):v for k, v in address_dict.items()}

In [221]:
fig = px.bar(punk_db['skin_tone'].value_counts().rename_axis('skin color').reset_index(name='# of punks'),
             x="skin color", y="# of punks", color="skin color", title="Skin Color Counts")

fig.write_image(f"{IMG_PATH}/analysis/fig1.png")
fig.show()

In [222]:
fig = px.bar(punk_db['gender'].value_counts().rename_axis('gender').reset_index(name='# of punks'),
             x="gender", y="# of punks", color="gender", title="Gender Counts")

fig.write_image(f"{IMG_PATH}/analysis/fig2.png")
fig.show()

In [223]:
skin_tone_prices = tx_db.groupby(['date', 'skin_tone']).agg({"eth_price": ["median"]}).unstack(1)
skin_tone_prices.columns = ['Albino', 'Dark', 'Light', 'Medium', 'Non-human']
skin_tone_prices = skin_tone_prices.reset_index()
skin_tone_prices = skin_tone_prices[skin_tone_prices['date'] > '2021-05-31']

overall_prices = tx_db[tx_db['skin_tone'] != 'Non-human'].groupby(['date']).agg({"eth_price": ["median"]})
overall_prices.columns = ['Overall']
overall_prices = overall_prices.reset_index()
overall_prices = overall_prices[overall_prices['date'] > '2021-05-31']

skin_tone_prices = skin_tone_prices.merge(overall_prices, on='date', how='left')

# skin_tone_prices.fillna(method='backfill', inplace=True)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=skin_tone_prices['date'], y=skin_tone_prices['Overall'], name="Overall", line=dict(width=1.3, color='#0077b6', dash='dot')))
fig.add_trace(go.Scatter(x=skin_tone_prices['date'], y=skin_tone_prices['Albino'], name="Albino", line=dict(width=1.3, color='#ffba08')))
fig.add_trace(go.Scatter(x=skin_tone_prices['date'], y=skin_tone_prices['Light'], name="Light", line=dict(width=1.3, color='#f48c06')))
fig.add_trace(go.Scatter(x=skin_tone_prices['date'], y=skin_tone_prices['Medium'], name="Medium", line=dict(width=1.3, color='#dc2f02')))
fig.add_trace(go.Scatter(x=skin_tone_prices['date'], y=skin_tone_prices['Dark'], name="Dark", line=dict(width=1.3, color='#9d0208')))

fig.update_yaxes(title_text="Price (ETH)")
fig.update_xaxes(title_text="Date")
fig.update_layout(
    # title="Median Price by Skin Tone vs Time (after 2021)",
    autosize=False,
    width=1500,
    height=600,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=20, r=20, t=20, b=20),
    # legend={'itemsizing': 'constant'},
    )

fig.update_xaxes(showline=True, linewidth=2, linecolor='#adb5bd', gridcolor='#adb5bd')
fig.update_yaxes(showline=True, linewidth=2, linecolor='#adb5bd', gridcolor='#adb5bd')

fig.write_image(f"{IMG_PATH}/analysis/fig3.png")
fig.show()

In [224]:
gender_prices = tx_db.groupby(['date', 'gender']).agg({"eth_price": ["median"]}).unstack(1)
gender_prices.columns = ['Female', 'Male']
gender_prices = gender_prices.reset_index()
gender_prices = gender_prices[gender_prices['date'] > '2020-12-31']

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=gender_prices['date'], y=gender_prices['Female'], name="Female", line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=gender_prices['date'], y=gender_prices['Male'], name="Male", line=dict(width=1.5)))

fig.update_yaxes(title_text="Price (ETH)")
fig.update_xaxes(title_text="Date")
fig.update_layout(
    title="Median Price by Gender vs Time (after 2021)",
    autosize=False,
    width=1500,
    height=500,
    plot_bgcolor='rgba(0,0,0,0.03)',
    )

fig.write_image(f"{IMG_PATH}/analysis/fig4.png")
fig.show()

In [225]:
def price_remove_25(df):
    return np.mean(df['eth_price'])
    # price25 = df.describe()['eth_price']['25%']
    # price75 = df.describe()['eth_price']['75%']
    # return np.mean(df[(df['eth_price'] > price25) & (df['eth_price'] < price75)]['eth_price'])

skin_tone_25 = pd.DataFrame(tx_db[(tx_db['date'] > '2020-12-31') & (tx_db['skin_tone'] != 'Non-human')].groupby('skin_tone').apply(price_remove_25)).T
skin_tone_25 = skin_tone_25[['Albino', 'Light', 'Medium', 'Dark']]

fig = go.Figure(data=[go.Bar(
    x=skin_tone_25.columns, 
    y=skin_tone_25.loc[0],
    text=skin_tone_25.loc[0],
    textposition='auto',),])

fig.update_layout(
    title="Median Price by Skin Tone (removing top and least 25%; after 2021)",
    autosize=False,
    width=1000,
    height=500,
    plot_bgcolor='rgba(0,0,0,0.03)',
    xaxis=dict(title='Skin Tone'),
    yaxis=dict(title='Price (ETH)'),
)

fig.update_traces(texttemplate='%{text:.5s}', textposition='outside')

fig.write_image(f"{IMG_PATH}/analysis/fig5.png")
fig.show()

In [226]:
gender_25 = tx_db[(tx_db['date'] > '2020-12-31')].groupby('gender').apply(price_remove_25)

fig = go.Figure(data=[go.Bar(
    x=gender_25.index, 
    y=gender_25.values,
    text=gender_25.values,
    textposition='auto',),])

fig.update_layout(
    title="Median Price by Gender (removing top and least 25%; after 2021)",
    autosize=False,
    width=600,
    height=500,
    plot_bgcolor='rgba(0,0,0,0.03)',
    xaxis=dict(title='Gender'),
    yaxis=dict(title='Price (ETH)'),
)

fig.update_traces(texttemplate='%{text:.5s}', textposition='outside')

fig.write_image(f"{IMG_PATH}/analysis/fig6.png")
fig.show()

In [227]:
tx_db_2021 = tx_db[tx_db['date'] > '2020-12-31']

tx_db_2021_human = tx_db_2021[tx_db_2021['skin_tone'] != 'Non-human']
tx_db_2021_human = tx_db_2021_human[(tx_db_2021_human['eth_price'] < 500) & (tx_db_2021_human['eth_price'] > 0)]
tx_db_2021_human

,date,from,to,eth_price,punk_id,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
9737,2021-01-01,1173,6060,11.44,4444,Human,Male,Dark,3,"['Normal Beard Black', 'Shaved Head', 'Mole']",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
6508,2021-01-02,3111,1211,5.30,9683,Human,Male,Dark,2,"['Goat', 'Top Hat']",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
9740,2021-01-02,6287,2234,5.47,9340,Human,Male,Light,3,"['Luxurious Beard', 'Mohawk Thin', 'Earring']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
9739,2021-01-02,1293,5236,4.97,3410,Human,Male,Light,2,"['Shadow Beard', 'Headband']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
8285,2021-01-02,412,4715,5.45,4861,Human,Male,Light,2,"['Normal Beard Black', 'Mohawk Thin']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...,...,...,...
12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,4,"['Cigarette', 'Mustache', 'Earring', 'Crazy Ha...",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,2,"['Clown Eyes Green', 'Frumpy Hair']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,3,"['Frumpy Hair', 'Gold Chain', 'Small Shades']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,4,"['Green Eye Shadow', 'Half Shaved', 'Earring',...",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...


In [228]:
tx_db_2021 = tx_db[tx_db['date'] > '2020-12-31']

tx_db_2021_human = tx_db_2021[tx_db_2021['skin_tone'] != 'Non-human']

In [229]:
def get_grouped_box_data(skin_tone):
    all_li = list(tx_db_2021_human[(tx_db_2021_human['skin_tone'] == skin_tone)]['eth_price'])
    dark_li = [
        list(tx_db_2021_human[(tx_db_2021_human['skin_tone'] == skin_tone) & (tx_db_2021_human['attr_count'] == attr_count)]['eth_price']) 
        for attr_count in [1, 2, 3, 4, 5]]
    
    all_attribute_count_list = ['All'] * len(all_li)
    dark_attribute_count_list = [
        len(dark_li[i])
        for i in range(len(dark_li))
    ]

    dark_li = all_li + list(chain(*dark_li))

    dark_attribute_count_list = [
        [f'{attr_count} attributes'] * dark_attribute_count_list[attr_count-1]
        for attr_count in [1, 2, 3, 4, 5]
    ]

    dark_attribute_count_list = all_attribute_count_list + list(chain(*dark_attribute_count_list))

    return dark_li, dark_attribute_count_list

dark_li, dark_attribute_count_list = get_grouped_box_data('Dark')
medium_li, medium_attribute_count_list = get_grouped_box_data('Medium')
light_li, light_attribute_count_list = get_grouped_box_data('Light')
albino_li, albino_attribute_count_list = get_grouped_box_data('Albino')

In [230]:
fig = go.Figure()

fig.add_trace(go.Box(
    x=dark_li,
    y=dark_attribute_count_list,
    name='Dark',
    marker_color='#9d0208'
))
fig.add_trace(go.Box(
    x=medium_li,
    y=medium_attribute_count_list,
    name='Medium',
    marker_color='#dc2f02'
))
fig.add_trace(go.Box(
    x=light_li,
    y=light_attribute_count_list,
    name='Light',
    marker_color='#f48c06'
))
fig.add_trace(go.Box(
    x=albino_li,
    y=albino_attribute_count_list,
    name='Albono',
    marker_color='#ffba08'
))

fig.update_layout(
    xaxis=dict(title='Price (ETH)', zeroline=False),
    boxmode='group',
    height=1200,
    width=1500,
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.update_xaxes(showline=True, linewidth=2, linecolor='#adb5bd', gridcolor='#adb5bd')

fig.update_traces(orientation='h') # horizontal box plots
fig.write_image(f"{IMG_PATH}/analysis/fig7.png")
fig.show()

In [231]:
tx_db_2021_human

,date,from,to,eth_price,punk_id,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
9737,2021-01-01,1173,6060,11.44,4444,Human,Male,Dark,3,"['Normal Beard Black', 'Shaved Head', 'Mole']",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
6508,2021-01-02,3111,1211,5.30,9683,Human,Male,Dark,2,"['Goat', 'Top Hat']",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
9740,2021-01-02,6287,2234,5.47,9340,Human,Male,Light,3,"['Luxurious Beard', 'Mohawk Thin', 'Earring']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
9739,2021-01-02,1293,5236,4.97,3410,Human,Male,Light,2,"['Shadow Beard', 'Headband']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
8285,2021-01-02,412,4715,5.45,4861,Human,Male,Light,2,"['Normal Beard Black', 'Mohawk Thin']",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...,...,...,...
12450,2022-07-26,980,1939,94.00,9099,Human,Male,Light,4,"['Cigarette', 'Mustache', 'Earring', 'Crazy Ha...",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
17440,2022-07-26,5318,4950,69.00,4430,Human,Female,Albino,2,"['Clown Eyes Green', 'Frumpy Hair']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
17613,2022-07-26,3002,1044,94.90,5600,Human,Male,Albino,3,"['Frumpy Hair', 'Gold Chain', 'Small Shades']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
13214,2022-07-27,3096,1028,69.69,6221,Human,Female,Dark,4,"['Green Eye Shadow', 'Half Shaved', 'Earring',...",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...


In [232]:
fig = go.Figure(data=[go.Bar(
    x=['Dark', 'Medium', 'Light', 'Albino'], 
    y=[41, 25, 16, 5],
    text=['41 (47%)', '25 (29%)', '16 (18%)', '5 (6%)'],
    textposition='auto',
    marker_color=['#9d0208', '#dc2f02', '#f48c06', '#ffba08']
    ),
])

fig.update_layout(
    # title="Median Price by Gender (removing top and least 25%; after 2021)",
    autosize=False,
    width=600,
    height=400,
    plot_bgcolor='rgba(0,0,0,0.03)',
    xaxis=dict(title='Skin Tone'),
    yaxis=dict(title='Number of Attributes'),
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.update_traces(textposition='inside')

fig.write_image(f"{IMG_PATH}/analysis/fig8.png")
fig.show()

In [250]:
tx_db

,date,from,to,eth_price,punk_id,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
0,2017-06-23,1406,1218,0.030,6548,Human,Male,Albino,4,"['Front Beard', 'Earring', 'Do-rag', 'Clown Ey...",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
30,2017-06-23,1406,1218,0.116,8781,Human,Male,Light,4,"['Frown', 'Big Shades', 'Shadow Beard', 'Knitt...",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
29,2017-06-23,1406,166,0.010,3134,Human,Male,Medium,3,"['Gold Chain', 'Crazy Hair', 'Regular Shades']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
28,2017-06-23,166,4722,0.100,5056,Human,Male,Albino,4,"['Beanie', 'Luxurious Beard', 'Earring', 'VR']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
26,2017-06-23,1406,1218,0.060,6208,Human,Male,Medium,2,"['Shadow Beard', 'Earring']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...,...,...,...
12450,2022-07-26,980,1939,94.000,9099,Human,Male,Light,4,"['Cigarette', 'Mustache', 'Earring', 'Crazy Ha...",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
17440,2022-07-26,5318,4950,69.000,4430,Human,Female,Albino,2,"['Clown Eyes Green', 'Frumpy Hair']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
17613,2022-07-26,3002,1044,94.900,5600,Human,Male,Albino,3,"['Frumpy Hair', 'Gold Chain', 'Small Shades']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
13214,2022-07-27,3096,1028,69.690,6221,Human,Female,Dark,4,"['Green Eye Shadow', 'Half Shaved', 'Earring',...",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...


In [283]:
traders = pd.DataFrame(tx_db.reset_index().groupby('from')['index'].count()).reset_index()
traders.columns = ['id', 'count']
traders

,id,count
0,0,1
1,1,2
2,2,2
3,3,1
4,6,1
...,...,...
3684,6431,1
3685,6433,1
3686,6434,4
3687,6435,1


In [291]:
address_db = pd.DataFrame.from_dict(address_dict, orient='index', columns=['address']).reset_index()
address_db.columns = ['id', 'address']

traders = pd.DataFrame(tx_db.reset_index().groupby('from')['index'].count()).reset_index()
traders.columns = ['id', 'count']
address_db = address_db.merge(traders, on='id', how='left')

traders = pd.DataFrame(tx_db.reset_index().groupby('to')['index'].count()).reset_index()
traders.columns = ['id', 'count']
address_db = address_db.merge(traders, on='id', how='left')

address_db.fillna(0, inplace=True)

address_db['count'] = address_db['count_x'] + address_db['count_y']
address_db.drop(['count_x', 'count_y'], axis=1, inplace=True)

address_db.sort_values('count', ascending=False, inplace=True)

address_db

,id,address,count
3096,3096,\x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,997.0
1303,1303,\x53ede7cae3eb6a7d11429fe589c0278c9acbe21a,768.0
1892,1892,\xd387a6e4e84a6c86bd90c158c6028a58cc8ac459,600.0
5223,5223,\x00d7c902fbbcd3c9db2da80a439c94486c50eb81,396.0
1291,1291,\x269616d549d7e8eaa82dfb17028d0b212d11232a,371.0
...,...,...,...
4188,4188,\xf1d2d8b848c7c76218e311b9c2f6f76905059133,1.0
4189,4189,\x03fbbfe27fae9bd53dbc5b5c7fc0504c9b3dd0b1,1.0
4190,4190,\x2cd8623df45c798e9605696d5f6b81fac0a316c0,1.0
4191,4191,\xf9dde1fed9a7ff50fdcfa72b8d9e0d4c7609d532,1.0


In [300]:
address_db.iloc[:10]

,id,address,count
3096,3096,\x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,997.0
1303,1303,\x53ede7cae3eb6a7d11429fe589c0278c9acbe21a,768.0
1892,1892,\xd387a6e4e84a6c86bd90c158c6028a58cc8ac459,600.0
5223,5223,\x00d7c902fbbcd3c9db2da80a439c94486c50eb81,396.0
1291,1291,\x269616d549d7e8eaa82dfb17028d0b212d11232a,371.0
3392,3392,\x577ebc5de943e35cdf9ecb5bbe1f7d7cb6c7c647,303.0
345,345,\x0e9aed5c7721c642a032812c2c4816f7d6cb87d7,266.0
2292,2292,\x6f4a2d3a4f47f9c647d86c929755593911ee91ec,247.0
4791,4791,\x6611fe71c233e4e7510b2795c242c9a57790b376,236.0
2642,2642,\x6639c089adfba8bb9968da643c6be208a70d6daa,196.0


In [293]:
count_sum = address_db['count'].sum()
count_sum

35652.0

In [299]:
count_sum_100 = address_db.iloc[:643]['count'].sum()
count_sum_100

23159.0

In [264]:
pd.DataFrame(tx_db.reset_index().groupby('to')['index'].count())

,index
to,
0,1
1,2
2,2
4,1
5,1
...,...
6432,1
6433,1
6434,3


In [253]:
tx_db.reset_index()

,index,date,from,to,eth_price,punk_id,type,gender,skin_tone,attr_count,attributes,skin_tone_color,img_url
0,0,2017-06-23,1406,1218,0.030,6548,Human,Male,Albino,4,"['Front Beard', 'Earring', 'Do-rag', 'Clown Ey...",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
1,30,2017-06-23,1406,1218,0.116,8781,Human,Male,Light,4,"['Frown', 'Big Shades', 'Shadow Beard', 'Knitt...",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
2,29,2017-06-23,1406,166,0.010,3134,Human,Male,Medium,3,"['Gold Chain', 'Crazy Hair', 'Regular Shades']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
3,28,2017-06-23,166,4722,0.100,5056,Human,Male,Albino,4,"['Beanie', 'Luxurious Beard', 'Earring', 'VR']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
4,26,2017-06-23,1406,1218,0.060,6208,Human,Male,Medium,2,"['Shadow Beard', 'Earring']",#DB9065,https://www.larvalabs.com/cryptopunks/cryptopu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17821,12450,2022-07-26,980,1939,94.000,9099,Human,Male,Light,4,"['Cigarette', 'Mustache', 'Earring', 'Crazy Ha...",#F2A359,https://www.larvalabs.com/cryptopunks/cryptopu...
17822,17440,2022-07-26,5318,4950,69.000,4430,Human,Female,Albino,2,"['Clown Eyes Green', 'Frumpy Hair']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
17823,17613,2022-07-26,3002,1044,94.900,5600,Human,Male,Albino,3,"['Frumpy Hair', 'Gold Chain', 'Small Shades']",#F2DC5D,https://www.larvalabs.com/cryptopunks/cryptopu...
17824,13214,2022-07-27,3096,1028,69.690,6221,Human,Female,Dark,4,"['Green Eye Shadow', 'Half Shaved', 'Earring',...",#A4031F,https://www.larvalabs.com/cryptopunks/cryptopu...
